In [1]:
import torch
from pycocotools.coco import COCO
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import time
import os
import matplotlib.pyplot as plt
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import sys
from model_new import Text2BBoxesModel
from dataset import get_data_loader_and_cats


dataType='val2017'
dataDir = './Datasets/coco/images/{}/'.format(dataType)
annFile_Detection ='./Datasets/coco/annotations/instances_{}.json'.format(dataType)
annFile_Caption ='./Datasets/coco/annotations/captions_{}.json'.format(dataType)
batch_size = 256
coco = COCO(annFile_Detection)
coco_captions = COCO(annFile_Caption)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
embed_size = 768
epochs = 30
learning_rate = 1e-3
num_mixtures = 5
beta1 = 0.9
beta2 = 0.999
ckpt_file_path = "./outputs/checkpoint_latest.ckpt"
print(device)

loading annotations into memory...
Done (t=0.48s)
creating index...
index created!
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
cpu


In [2]:
dataloader, total_categories = get_data_loader_and_cats(annFile_Caption, annFile_Detection, batch_size, True)
start = time.time()
num_classes = total_categories[max(total_categories, key=total_categories.get)] + 1
model = Text2BBoxesModel(embed_size, total_categories, batch_size, device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, betas=(beta1, beta2))

# mdn_xy_model = MDN(2, num_mixtures)
# mdn_wh_model = MDN(2, num_mixtures)
# if(os.path.exists(ckpt_file_path)) :
#     checkpoint = torch.load(ckpt_file_path, map_location=device)
#     model.load_state_dict(checkpoint['model_state_dict'])
#     optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#     print("Model loaded with training up to {} epochs and {} minibatches".format(checkpoint['epoch'], checkpoint['count']))
    

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.62s)
creating index...
index created!


In [3]:
model = model.to(device)
for epoch in range(epochs) : 
    count = 0
    for _, captions_mini_batch, bboxes, categories, lengths in dataloader :
        captions_mini_batch = captions_mini_batch.to(device)
        bboxes = bboxes.to(device)
        categories = categories.to(device)
        loss_what = 0
        loss_where = 0
        optimizer.zero_grad()
        max_length = max(lengths)
        lengths = torch.Tensor(lengths)
        outputs = model(captions_mini_batch, max_length)
        

        categories = categories.permute(1, 0)
        seqlen = categories.shape[0] 
        pred_labels = outputs[:,:,:-4]
        pred_bboxes_xy = outputs[:,:,-4:-2]
        pred_bboxes_wh = outputs[:,:,-2:]
        
#         theta_xy = mdn_xy_model(pred_bboxes_xy)
#         theta_wh = mdn_wh_model(pred_bboxes_wh)
#         print(theta_xy.shape, theta_wh.shape)
        for i in range(0,seqlen) :
            loss_what += F.cross_entropy(pred_labels[i, :, :], categories[i, :].long())
        loss_what /= seqlen
        loss_what.backward()
        optimizer.step()
        count += 1            
        if(count % 100 == 0) :
            checkpoint = {'model_state_dict': model.state_dict(),
                          'optimizer_state_dict' : optimizer.state_dict(),
                          'epoch'  : epoch,
                          'count'  : count}
            torch.save(checkpoint, ckpt_file_path)
            print("After processing {} minibatches in epoch {} , loss is {}".format(count, epoch, loss))
        if(epoch == 0) :
            break


    print("Loss after epoch {} : {}".format(epoch, loss))

/Users/umangsharma/Documents/Projects/DeepLearning/Text2Bboxes/MDN.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pi = F.softmax(self.pi_layer(input))


====> torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5])
====> torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5])
====> torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5])
====> torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5])
====> torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5])
====> torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5])
====> torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5])
====> torch.Size([256, 5]) torch.Size([256, 5]) torch.Size([256, 5]) torch.S

NameError: name 'loss' is not defined

In [16]:
print("Predictions on validation sets")
with torch.no_grad() :
        dataType = 'val2017'
        dataDir = './Datasets/coco/images/{}/'.format(dataType)
        annFile_Detection ='./Datasets/coco/annotations/instances_{}.json'.format(dataType)
        annFile_Caption ='./Datasets/coco/annotations/captions_{}.json'.format(dataType)
        val_dataloader, total_categories = get_data_loader_and_cats(annFile_Caption, annFile_Detection, batch_size, True)
        val_loss = 0
        for caption_txt, caption, bboxes, categories, lengths in val_dataloader :
            caption = caption.to(device)
            bboxes = bboxes.to(device)
            categories = categories.to(device)
            max_length = max(lengths)
            pred_labels = model(caption, max_length)
            vals, idx = torch.max(pred_labels, 2)
            categories = categories.permute(1, 0)
            seqlen = categories.shape[0] 
            for i in range(0,seqlen) :
                val_loss += F.cross_entropy(pred_labels[i, :, :], categories[i, :].long())
            val_loss /= seqlen
            break
        key_list = list(total_categories.keys()) 
        val_list = list(total_categories.values()) 
        print("Loss on validaton set : {}".format(val_loss))
        print("Predicted labels shape : {}, Ground truth labels shape {}".format(idx.shape, categories.shape))
        # Print a sample of 10 captions from the val set
        for i in range(10) :
            print("Text input : {}".format(caption_txt[i]))            
            print("Predicted labels:")
            print([key_list[val_list.index(j)] for j in idx[:,i].tolist()])
            print("Ground truth labels:")
            print([key_list[val_list.index(j)] for j in categories[:,i].tolist()])

            
            

Predictions on validation sets
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.38s)
creating index...
index created!
Loss on validaton set : 0.5318634510040283
Predicted labels shape : torch.Size([47, 256]), Ground truth labels shape torch.Size([47, 256])
Text input : An old fashioned kitchen with the light on. 
Predicted labels:
['start', 'refrigerator', 'oven', 'oven', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad']
Ground truth labels:
['start', 'refrigerator', 'chair', 'dining table', 'bowl', 'bowl', 'sink', 'cup', 'spoon', 'chair', 'banana', 'cup', 'end', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'p

In [4]:
dataType = 'val2017'
dataDir = './Datasets/coco/images/{}/'.format(dataType)
annFile_Detection ='./Datasets/coco/annotations/instances_{}.json'.format(dataType)
annFile_Caption ='./Datasets/coco/annotations/captions_{}.json'.format(dataType)
val_dataloader, total_categories = get_data_loader_and_cats(annFile_Caption, annFile_Detection, batch_size, True)
for caption, bboxes, categories, lengths in val_dataloader :
    max_length = max(lengths)
    with torch.no_grad() :
        pred_labels = model(caption, max_length)
        vals, idx = torch.max(pred_labels, 2)
        categories = categories.permute(1, 0)
        seqlen = categories.shape[0] 
        val_loss = 0
        print(pred_labels.shape, categories.shape)
        for i in range(0,seqlen) :
            val_loss += F.cross_entropy(pred_labels[i, :, :], categories[i, :].long())
        val_loss /= seqlen
        print("Loss on validaton set : {}".format(val_loss))
        print("Predicted labels shape : {}, Ground truth labels shape {}".format(idx.shape, categories.shape))
        print("Predicted labels {}, Ground truth labels {}".format(total_categories[idx[:,0]], total_categories[categories[:,0]]))


loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
loading annotations into memory...
Done (t=0.36s)
creating index...
index created!


ValueError: too many values to unpack (expected 4)

In [6]:
start_tensor = self.index_to_one_hot(self.SOS)
label = start_tensor.repeat(self.batch_size,1)

torch.Size([256, 1])


RuntimeError: invalid argument 3: out of range at /Users/distiller/project/conda/conda-bld/pytorch_1565272526878/work/aten/src/TH/generic/THTensor.cpp:370